In [15]:
import pandas as pd
import numpy as np
import re

In [21]:
df = pd.read_csv('data/ADAS_ADNIGO23_17Apr2024.csv')

In [25]:
rids = df['RID'].unique()
rids = np.array(rids)
rids.size

1908

In [9]:
np.savetxt('rids.txt', rids, fmt='%d')

In [10]:
def get_rid_viscode(filename):
    pattern = r'sub-ADNI\d+S(\d{4})_ses-(M\d{3})'
    match = re.search(pattern, filename)

    if match:
        rid = match.group(1)
        viscode = match.group(2)
        return rid, viscode        
    else:
        print("Pattern not found in the filename.")
        return None

In [26]:
# file_rids = []

with open('files.txt', 'r') as file:
        file_rids = [int(get_rid_viscode(name)[0]) for name in [line.strip() for line in file.readlines()]]

In [29]:
i = 0
for rid in rids:
    if rid in file_rids:
        # print('yep')
        i += 1
    else:
        print('Nope, missing: ', rid)
print('Data: ', i)

Nope, missing:  376
Nope, missing:  1117
Nope, missing:  1326
Nope, missing:  618
Nope, missing:  292
Nope, missing:  2002
Nope, missing:  2007
Nope, missing:  622
Nope, missing:  2011
Nope, missing:  548
Nope, missing:  2003
Nope, missing:  684
Nope, missing:  667
Nope, missing:  2026
Nope, missing:  2037
Nope, missing:  698
Nope, missing:  538
Nope, missing:  1414
Nope, missing:  2031
Nope, missing:  702
Nope, missing:  2047
Nope, missing:  545
Nope, missing:  2027
Nope, missing:  2055
Nope, missing:  1417
Nope, missing:  135
Nope, missing:  722
Nope, missing:  459
Nope, missing:  558
Nope, missing:  800
Nope, missing:  778
Nope, missing:  2058
Nope, missing:  420
Nope, missing:  657
Nope, missing:  2052
Nope, missing:  2071
Nope, missing:  824
Nope, missing:  2061
Nope, missing:  2
Nope, missing:  2070
Nope, missing:  2072
Nope, missing:  2083
Nope, missing:  668
Nope, missing:  658
Nope, missing:  887
Nope, missing:  519
Nope, missing:  553
Nope, missing:  845
Nope, missing:  8
Nop